In [2]:
#importar librerias
import pandas as pd
import numpy as np

In [4]:
#leer bases de datos
cronicos = pd.read_csv('databases/RETO_df_cronicos.csv',low_memory=False)
egresos= pd.read_csv('databases/RETO_df_egresos.csv')   
usuarios = pd.read_csv('databases/RETO_df_usuarios.csv',low_memory=False)

In [5]:
usuarios.head()

,MES,YEAR,NRODOC,SEXO,FECHA NACIMIENTO,EDAD,DEPARTAMENTO,MUNICIPIO,BARRIO,FECHA INICIO AL PGP,FECHA PRIMERA CLASE FUNCIONAL,PRIMERA CLASE FUNCIONAL,FECHA ÚLTIMA CLASE FUNCIONAL,ÚLTIMA CLASE FUNCIONAL,CICLO_VITAL,QUINQUENIO
0,ABRIL,2017,306221,F,1938-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRIL,2017,306222,F,1974-07-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABRIL,2017,306223,F,1930-08-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABRIL,2017,306224,F,1979-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABRIL,2017,306225,M,1943-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## filtro inicial 

In [42]:
#crear tabla nueva con los usuarios que tienen un ciclo vital en vejez
vejez = usuarios[usuarios["CICLO_VITAL"] == "Vejez"]
vejez.shape
#unir las 3 bases de datos
df1 = pd.merge(vejez, egresos, on='NRODOC', how='inner').merge(cronicos, on='NRODOC', how='inner')
#crear tabla nueva con la base de datos egresos con MODALIDAD CONTRATO = PGP
df1= df1[df1["MODALIDAD CONTRATO"] == "PGP"]
df1.shape

(549854, 370)

In [41]:
#convertir a formato "2017-04-30 00:00:00"  fecha FECHA INICIO AL PGP 
df1["FECHA INICIO AL PGP"] = pd.to_datetime(df1["FECHA INICIO AL PGP"], format='%Y-%m-%d %H:%M:%S')


#quiere ver el mayor y minimo de FECHA INICIO AL PGP
print(df1["FECHA INICIO AL PGP"].max())
print(df1["FECHA INICIO AL PGP"].min())

2022-02-03 00:00:00
2017-04-01 00:00:00


In [43]:
df1["NRODOC"].value_counts()

NRODOC
308373    3744
307942    3360
307148    3136
309697    2592
306404    2304
          ... 
311672       3
313763       2
313282       2
312437       2
309859       2
Name: count, Length: 1830, dtype: int64

In [49]:
# ver columnas nulas mayores a 0 en la base de datos df1 en porcentaje 
nulos=df1.isnull().sum()/len(df1)*100
# ver los nulos vayores a 80 de la base nulos
nulos[nulos>80]
# y eliminarlos de la base de datos df1
df1=df1.drop(columns=nulos[nulos>80].index)

In [58]:
## ver columnas nulas mayores a 0 en la base de datos df1 
nulos=df1.isnull().sum()/len(df1)*100
# guardar en la base de datos nulos y filtrar los valores mayores a 0 
nulos=nulos[nulos>0]





In [60]:
# se tiene que observar 90 columnas
# crear un for con un value count para la base nulos
for i in nulos.index:
    print(df1[i].value_counts())
    print("")










FECHA NACIMIENTO_x
1940-07-29 00:00:00    2106
1949-09-11 00:00:00    1792
1929-11-02 00:00:00    1764
1937-03-03 00:00:00    1296
1941-02-07 00:00:00    1260
                       ... 
1940-01-10 00:00:00       2
1955-03-03 00:00:00       2
1962-06-02 00:00:00       2
1930-05-04 00:00:00       1
1928-06-30 00:00:00       1
Name: count, Length: 1398, dtype: int64

DEPARTAMENTO
ANTIOQUIA    309739
Name: count, dtype: int64

MUNICIPIO
MEDELLIN                246964
BELLO                    26398
ITAGUI                   16158
ENVIGADO                  8590
SAN ANTONIO DE PRADO      6410
SABANETA                  2056
LA ESTRELLA                854
SAN CRISTOBAL              846
VEREDA YARUMALITO          504
COPACABANA                 288
EL PORVENIR                216
VEREDA LA MARIA            126
SANTA ELENA                 90
SAN FELIX                   72
Name: count, dtype: int64

BARRIO
MEDELLIN                  44982
MANRIQUE                  22107
ARANJUEZ                  1555

FECHA NACIMIENTO_x     43.667410
DEPARTAMENTO           43.668865
MUNICIPIO              43.699237
BARRIO                 19.857999
FECHA INICIO AL PGP    14.614971
                         ...    
NombreDiagnostico1     57.929559
Diagnostico2           67.871653
NombreDiagnostico2     67.871653
Diagnostico3           73.159421
NombreDiagnostico3     73.159421
Length: 90, dtype: float64